In [1]:
using FuzzifiED
FuzzifiED.SilentStd = true
FuzzifiED.ElementType = Float64
≊(x, y) = abs(x - y) < eps(Float32)
using LinearAlgebra
using Optim
using WignerSymbols
using SpecialFunctions
using PrettyTables
using Plots
using JLD2

const σ1 = [  1  0 ;  0  0 ]
const σ2 = [  0  0 ;  0  1 ]
const σx = [  0  1 ;  1  0 ]

2×2 Matrix{Int64}:
 0  1
 1  0

# function

In [2]:
function my_pochhammer(x, n)
    result = 1.0
    for i in 0:n-1
        result *= (x + i)
    end
    return result
end

function mymod(n, m)
   return n - Int(floor(n/m))*m 
end

function Delta(nm, l, m, mat; nf=2)
    no = nf * nm
    s = .5 * (nm - 1)
    ne = div(no, 2)
    tms = Vector{Term}(undef, 0)
    for m1 = 0 : nm-1
        m1r = m1 - s
        m2r = m - (m1 - s)
        m2 = m2r + s
        if (m2r < -s - 0.01 || m2r > s + 0.01) continue end
        for f1 = 1 : nf
            # o1 = m1 + 1 + (f1-1)*nm
            o1 = m1 * nf + f1
            for f2 = 1 : nf
                o2 = m2*nf + f2
                
                val = (-1)^m * sqrt(2*l+1) * wigner3j(s, s, l, m1r, m2r, -m) * mat[f1, f2]
                push!(tms, Term(val, [0, o1, 0, o2]))
            end
        end
    end
    
    return tms
end

function fourF(nm, l, m, la, lb, mata, matb; nf=2)
    # Here we add an extra (-1)^lb factor compared to the notes
    tms = Vector{Term}(undef, 0)
    for ma = -la : la
        mb = ma - m
        if (mb < -lb - 0.01 || mb > lb + 0.01) continue end
        tms_temp1 = Delta(nm, la, ma, mata; nf=nf)
        tms_temp2 = Delta(nm, lb, mb, matb; nf=nf)
        val = (-1)^(lb+mb) * wigner3j(la, lb, l, ma, -mb, -m)
        tms +=  SimplifyTerms(val * tms_temp1' * tms_temp2)
        # tms +=  val * tms_temp1' * tms_temp2
        
    end
    return SimplifyTerms(tms)
end

function nlm(nm, l, m, mat; nf=2)
    # didn't include 1/sqrt(4pi)
    no = nf * nm
    s = .5 * (nm - 1)
    ne = div(no, 2)
    tms = Vector{Term}(undef, 0)
    for m1 = 0 : nm-1
        m1r = m1 - s
        m2r = m1 - s - m
        m2 = m2r + s
        if (m2r < -s - 0.01 || m2r > s + 0.01) continue end
#         @show m1, m2
        for f1 = 1 : nf
            o1 = m1 * nf + f1
            for f2 = 1 : nf
                o2 = m2*nf + f2
                val = (2*s+1) * sqrt(2*l+1) * (-1)^(s+m1r) * wigner3j(s, s, l, m1r, -m2r, -m) * wigner3j(s, s, l, -s, s, 0) * mat[f1, f2]
                push!(tms, Term(val, [1, o1, 0, o2]))
            end            
        end
    end
    return tms
    
end

function scalar_nn(nm, l; mata=[1 0; 0 1], matb=[1 0; 0 1])
    tempt = Term[]
    for m in -l:l
        tempt += wigner3j(l, l, 0, m, -m, 0) * nlm(nm, l, m, mata) * nlm(nm, l, -m, matb)
    end
    return SimplifyTerms(tempt)
end

scalar_nn (generic function with 1 method)

In [3]:
function compute_state(nm)
ps_pot = [4.75, 1.]
# function cost(ps_pot)
qnd = [ 
    GetNeQNDiag(2 * nm), 
    GetLz2QNDiag(nm, 2) ]
qnf = [ 
    GetParityQNOffd(nm, 2, [2, 1], [-1, 1]), 
    GetFlavPermQNOffd(nm, 2, [2, 1]), 
    GetRotyQNOffd(nm, 2) ]

tms_hmt = SimplifyTerms(
    GetDenIntTerms(nm, 2, 0.5*ps_pot, [1 0; 0 1], [1 0; 0 1]) -
    GetDenIntTerms(nm, 2, 0.5*ps_pot, [1 0; 0 -1], [1 0; 0 -1]) +
#     GetDenIntTerms(nm, 2, 2 .* ps_pot, σ1, σ2) - 
    3.16 * GetPolTerms(nm, 2, σx) )
tms_l2 = GetL2Terms(nm, 2)
cfs = Confs(2 * nm, [nm, 0], qnd)

result = []

for P in [1], Z in [1, -1]
    bs = Basis(cfs, [P, Z, 0], qnf)
#     bs = Basis(cfs)
    hmt = Operator(bs, tms_hmt)
    hmt_mat = OpMat(hmt)
    # hmt_mat_full = MatrixFromOpMat(hmt_mat)
    # enrg, st = eigen(hmt_mat_full)
    enrg, st = GetEigensystem(hmt_mat, 40)

    l2 = Operator(bs, tms_l2)
    l2_mat = OpMat(l2)
    l2_val = [ st[:, i]' * l2_mat * st[:, i] for i in eachindex(enrg)]
    
    
    for i in eachindex(enrg)
        push!(result, [round.(real(enrg[i]), digits = 6), 
                round.(abs(real((sqrt(4*l2_val[i]+1)-1)/2)), digits = 6), P, Z, st[:,i]])
    end
end
    
sort!(result, by = st -> real(st[1]))
enrg_0 = result[1][1]
enrg_T = filter(st -> st[2] ≊ 2, result)[2][1]
result_dim = [ [ 3 * (st[1] - enrg_0) / (enrg_T - enrg_0) ; st] for st in result ];

return result_dim
    
end

compute_state (generic function with 1 method)

# compute

In [4]:
nm = 12
result = compute_state(nm);

qnd = [ 
    GetNeQNDiag(2 * nm), 
    GetLz2QNDiag(nm, 2) ]
qnf = [ 
    GetParityQNOffd(nm, 2, [2, 1], [-1, 1]), 
    GetFlavPermQNOffd(nm, 2, [2, 1]), 
    GetRotyQNOffd(nm, 2) ];

cfs = Confs(2 * nm, [nm, 0], qnd)
bs = Basis(cfs, [1, 1, 0], qnf)
bs_odd = Basis(cfs, [1, -1, 0], qnf);

In [5]:
σ = result[2]

6-element Vector{Any}:
   0.5236388387808408
 -16.510012
   0.0
   1
  -1
    [-4.9952840088774055e-8, 1.657242726831872e-8, 1.6572427273606212e-8, 3.8609743655177437e-19, 1.657242727158511e-8, -1.8114964207113248e-18, 3.1179497586047994e-18, -1.6572427272155923e-8, -5.7702147341632694e-8, 1.7536103394815396e-8  …  -0.029636600887532148, 0.06052448437833153, 0.01613105457259856, -0.02748032509901964, -0.03073899676087436, 0.05997288630412292, -0.030548437859166305, 0.05960570804828643, 0.059361650443709967, -0.138459166245873]

In [6]:
for run in 1:20
    l=0
    m=0
    opterm = nlm(nm, l, m, [1 0; 0 -1]) - 0.2 * nlm(nm, l, m, [0 -1; 1 0])
    bdag = OpMat(Operator(bs_odd, bs, SimplifyTerms(opterm'), sym_q=0))
    temp = bdag * σ[6]
    @show run, temp' * temp
end

(run, temp' * temp) = (1, 70.23709002345281)
(run, temp' * temp) = (2, 70.01470725959904)
(run, temp' * temp) = (3, 80.77632501519597)
(run, temp' * temp) = (4, 70.05104691426834)
(run, temp' * temp) = (5, 70.17973722509427)
(run, temp' * temp) = (6, 70.5480683839936)
(run, temp' * temp) = (7, 70.05072620898575)
(run, temp' * temp) = (8, 95.985607919884)
(run, temp' * temp) = (9, 81.37720908111275)
(run, temp' * temp) = (10, 80.09408111410258)
(run, temp' * temp) = (11, 97375.06548598067)
(run, temp' * temp) = (12, 71.2319896325936)
(run, temp' * temp) = (13, 107.63099386264024)
(run, temp' * temp) = (14, 71.20184681902388)
(run, temp' * temp) = (15, 70.05104691426834)
(run, temp' * temp) = (16, 70.17973722509427)
(run, temp' * temp) = (17, 81.82163893717004)
(run, temp' * temp) = (18, 117.32719008462155)
(run, temp' * temp) = (19, 131.09272820192373)
(run, temp' * temp) = (20, 77.85979867343649)


In [7]:
gs = result[1]
for run in 1:20
    l=0
    m=0
    opterm = nlm(nm, l, m, [1 0; 0 -1]) - 0.2 * nlm(nm, l, m, [0 -1; 1 0])
    bdag = OpMat(Operator(bs, bs_odd, SimplifyTerms(opterm'), sym_q=0))
    temp = bdag * gs[6]
    @show run, temp' * temp
end

(run, temp' * temp) = (1, 36.680227649942246)
(run, temp' * temp) = (2, 36.680227649942246)
(run, temp' * temp) = (3, 36.680227649942246)
(run, temp' * temp) = (4, 36.680227649942246)
(run, temp' * temp) = (5, 36.680227649942246)
(run, temp' * temp) = (6, 36.680227649942246)
(run, temp' * temp) = (7, 36.680227649942246)
(run, temp' * temp) = (8, 36.680227649942246)
(run, temp' * temp) = (9, 36.680227649942246)
(run, temp' * temp) = (10, 36.680227649942246)
(run, temp' * temp) = (11, 36.680227649942246)
(run, temp' * temp) = (12, 36.680227649942246)
(run, temp' * temp) = (13, 36.680227649942246)
(run, temp' * temp) = (14, 36.680227649942246)
(run, temp' * temp) = (15, 36.680227649942246)
(run, temp' * temp) = (16, 36.680227649942246)
(run, temp' * temp) = (17, 36.680227649942246)
(run, temp' * temp) = (18, 36.680227649942246)
(run, temp' * temp) = (19, 36.680227649942246)
(run, temp' * temp) = (20, 36.680227649942246)
